<a href="https://colab.research.google.com/github/ariwells2001/aqara-open-api/blob/main/authCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import json


# User information
account= 'ariwells2001@gmail.com'
password= 'iot12345'



atSign = '%40'
colonSign='%3A'
slashSign='%2F'

appID = '869584076517146624aa4c13'
appKey = '5at22qrvj0a5tfm8ls7s4jb6lqdnkm07'
keyID = 'K.944533654225899520'

response_type= 'code'
redirect_url= 'https://ariwells-redirect.herokuapp.com/aqararesponse'
state= 'abc'

client_id = 'client_id='+ appID + '&'
response_type = 'response_type=' + response_type + '&'
redirect_url='redirect_uri=' + (redirect_url.replace(':','%3A')).replace('/','%2F') + '&'
account='account=' + account.replace('@','%40') + '&'
password='password=' + password + '&'
state='state=' + state

url = "https://open-kr.aqara.com/v3.0/open/authorize"

payload = client_id+response_type+redirect_url+account+password+state

headers = {
  'Content-Type': 'application/x-www-form-urlencoded'
}


response = requests.post(url, headers=headers, data=payload)
response = json.loads(response.text)

authCode = response['result']['code']

print('authCode is {}'.format(authCode))

authCode is 954b49e671f6c4ed92a916ee7d4debf1


In [2]:
url_accesstoken = 'https://open-kr.aqara.com/v3.0/open/access_token'
redirect_url= 'https://ariwells-redirect.herokuapp.com/aqararesponse'

client_secret = 'client_secret=' + appKey + '&'
grant_type = 'grant_type=' + 'authorization_code' + '&' 
code = 'code=' + authCode + '&'
redirect_url='redirect_uri=' + (redirect_url.replace(':','%3A')).replace('/','%2F')

payload = client_id + client_secret + grant_type + code + redirect_url

response = requests.post(url_accesstoken, headers=headers, data=payload)
response = json.loads(response.text)
print (type(response))
print(response['access_token'])
accessToken = response['access_token']
refreshToken = response['refresh_token']

print('accessToken is {}'.format(accessToken))

<class 'dict'>
434289069d57324b798493c9c2a459e6
accessToken is 434289069d57324b798493c9c2a459e6


In [3]:
import time
import requests
import json
import hashlib


keyVar = {

    'AppidVar' : '869584076517146624aa4c13',
    'AppkeyVar' : '5at22qrvj0a5tfm8ls7s4jb6lqdnkm07',
    'KeyidVar' : 'K.944533654225899520',
    'AccesstokenVar' : accessToken,
    'RefreshtokenVar': refreshToken
}


def getKeyVar(myAccesstoken):
    
    
    currentUTC = str(round(time.time(),3))
    tTime = currentUTC.replace('.','')
    tAppid = keyVar['AppidVar']
    tAccesstoken = myAccesstoken
    tKeyid = keyVar['KeyidVar']
    tAppKey = keyVar['AppkeyVar']
    tNonce = tTime
    tPreSign = ""

    if tAccesstoken != None:
        tPreSign = "Accesstoken=" + tAccesstoken + "&"

    tempSign = tPreSign + "Appid=" + tAppid + "&" + "Keyid=" + tKeyid + "&" + "Nonce=" + tNonce + "&" + "Time=" + tTime + tAppKey
    tPreSign = tempSign.lower()
    tSign = str(hashlib.md5(tPreSign.encode()).hexdigest())

    resultKeyVar = {
        'Time': tTime,
        'Nonce': tNonce,
        'Sign': tSign,
            }
    print(resultKeyVar)
    return resultKeyVar

resultKeyVar = getKeyVar(keyVar['AccesstokenVar'])

keyVariables = {
        'SignVar' : resultKeyVar['Sign'],
        'TimeVar' : resultKeyVar['Time'],
        'NonceVar' : resultKeyVar['Nonce'],
        'AppidVar' : keyVar['AppidVar'],
        'AppkeyVar' : keyVar['AppkeyVar'],
        'KeyidVar' : keyVar['KeyidVar'],
        'AccesstokenVar' : keyVar['AccesstokenVar'],
        'RefreshtokenVar' : keyVar['RefreshtokenVar'],
        'urlVar' : 'https://open-kr.aqara.com/v3.0/open/api',
        'urlCodeAuth20' : 'https://open-kr.aqara.com/v3.0/open/authorize',
        'urlAccessTokenAuth20' : 'https://open-kr.aqara.com/v3.0/open/access_token'
    }

headers = {
    'Content-Type' : 'application/json;charset=utf-8',
    'Appid' : keyVariables['AppidVar'],
    'Accesstoken': keyVariables['AccesstokenVar'],
    'Keyid': keyVariables['KeyidVar'],
    'AppKey': keyVariables['AppkeyVar'],
    'Time': keyVariables['TimeVar'],
    'Nonce': keyVariables['NonceVar'],
    'Sign': keyVariables['SignVar']
    }

{'Time': '1645773128406', 'Nonce': '1645773128406', 'Sign': '60c5768abf765ec1841537ce8223adb1'}


In [4]:
payload = json.dumps({
  "intent": "query.position.info",
  "data": {
    "pageNum": 1,
    "pageSize": 5
  }
})

response = requests.post(keyVariables['urlVar'], headers=headers, data=payload)
response = json.loads(response.text)
positionId = response['result']['data'][0]['positionId']
print('positionId is {}'.format(positionId))

positionId is real1.943997533140799488
